<a href="https://colab.research.google.com/github/Ali7109/GoogleTimelineAnalyzer/blob/master/GoogleTimelineServer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install anvil-uplink

In [ ]:
import anvil.server
anvil.server.connect("server_IWPW4XPEAEIMFHCV2RA7WD3R-PPOUQU3YMTVCRYI2")

# Imports / Installs

In [ ]:
!pip install reverse-geocode

In [4]:
import json
import datetime
import pandas as pd
from geopy import distance
import anvil.media
import reverse_geocode

# Server Side



In [35]:
from IPython.utils.contexts import NoOpContext
# Server call to check server status
@anvil.server.callable
def output(message):
  return "" + message + " received"


# Server call to receieve and transfer file to preprocessing
@anvil.server.callable
def process_file(file):

  with anvil.media.TempFile(file) as f:
        TIMELINE_DATA = pd.read_json(f)
        return run_process(TIMELINE_DATA)

TRAVEL_DATA_READY = []


@anvil.server.callable
def show_travel_data():
  if len(TRAVEL_DATA_READY) != 0:

    return TRAVEL_DATA_READY
  else:
    return None

@anvil.server.callable
def calculate_travels(from_input, to_input):
  if len(TRAVEL_DATA_READY) != 0:

    print("------- Sub process: calculating travels ---------")

    count = 0
    for log in TRAVEL_DATA_READY:
      if log[0][2].upper() == from_input.upper() and log[1][2].upper() == to_input.upper():
        count += 1
    print(count)
    return count
  else:
    return None

@anvil.server.callable
def calculate_entries(to_input):
  if len(TRAVEL_DATA_READY) != 0:
    print("------- Sub process: calculating entries ---------")
    count = 0
    for log in TRAVEL_DATA_READY:
      if log[0][2].upper() != log[1][2].upper() and log[1][2].upper() == to_input.upper():
        count += 1
    return count
  else:
    return None


def run_process(TIMELINE_DATA):
  global TRAVEL_DATA_READY  # Use the global keyword

  try:
    print("-------Processing---------")
    pre_processed = preprocess(TIMELINE_DATA)
    reverse_geocoded_data = reverse_geocode_data(pre_processed)
    print("-------Processed---------")

    for log in reverse_geocoded_data:
      cc_from = log[0]['country_code']
      city_from = log[0]['city']
      country_from = log[0]['country']

      cc_to = log[1]['country_code']
      city_to = log[1]['city']
      country_to = log[1]['country']

      TRAVEL_DATA_READY.append(((cc_from, city_from, country_from), (cc_to, city_to, country_to)))
    return True
  except:
    return False

"""
[{'country_code': 'US', 'city': 'Byram', 'country': 'United States'}, {'country_code': 'US', 'city': 'Bridgeport', 'country': 'United States'}]
[{'country_code': 'US', 'city': 'Byram', 'country': 'United States'}, {'country_code': 'US', 'city': 'Bridgeport', 'country': 'United States'}]
[{'country_code': 'US', 'city': 'Byram', 'country': 'United States'}, {'country_code': 'US', 'city': 'Bridgeport', 'country': 'United States'}]
[{'country_code': 'US', 'city': 'Byram', 'country': 'United States'}, {'country_code': 'US', 'city': 'Bridgeport', 'country': 'United States'}]
[{'country_code': 'US', 'city': 'North Castle', 'country': 'United States'}, {'country_code': 'US', 'city': 'Bridgeport', 'country': 'United States'}]
[{'country_code': 'US', 'city': 'North Castle', 'country': 'United States'}, {'country_code': 'US', 'city': 'Westport', 'country': 'United States'}]
[{'country_code': 'US', 'city': 'North Castle', 'country': 'United States'}, {'country_code': 'US', 'city': 'Westport', 'country': 'United States'}]
[{'country_code': 'US', 'city': 'North Castle', 'country': 'United States'}, {'country_code': 'US', 'city': 'Westport', 'country': 'United States'}]
[{'country_code': 'US', 'city': 'North Castle', 'country': 'United States'}, {'country_code': 'US', 'city': 'Westport', 'country': 'United States'}]
[{'country_code': 'US', 'city': 'North Castle', 'country': 'United States'}, {'country_code': 'US', 'city': 'Westport', 'country': 'United States'}]


[('US', 'US'), ('US', 'US'), ('US', 'US'), ('US', 'US'), ('US', 'US'), ('US', 'US'), ('US', 'US'), ('US', 'US'), ('US', 'US'), ('US', 'US')]


"""

def preprocess(data):
  data = data["timelineObjects"]
  cleaned_travels = []

  for travel in data:

    if "activitySegment" in travel:
      fromLocation = travel["activitySegment"]["startLocation"]
      toLocation = travel["activitySegment"]["endLocation"]
      print(travel)
      cleaned_travel = {}


      fromLocation["latitudeE7"] = round(fromLocation["latitudeE7"]/10000000,2)
      fromLocation["longitudeE7"] = round(fromLocation["longitudeE7"]/10000000,2)
      toLocation["latitudeE7"] = round(toLocation["latitudeE7"]/10000000,2)
      toLocation["longitudeE7"] = round(toLocation["longitudeE7"]/10000000,2)

      cleaned_travel["from"] = (fromLocation["latitudeE7"], fromLocation["longitudeE7"])
      cleaned_travel["to"] = (toLocation["latitudeE7"], toLocation["longitudeE7"])
      cleaned_travels.append(cleaned_travel)
  return cleaned_travels

def reverse_geocode_data(data):
  travels = []

  for log in data:
    coordinates = log["from"], log["to"]
    travels.append(reverse_geocode.search(coordinates))

  return travels

#anvil.server.wait_forever()